In [9]:
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
import astropy.io.fits as pf
import sys, glob

#thedir='/project/bootcamp/cslage/ats_data/20190416/ptc_kirk_config/' # Current voltages
#thedir='/project/bootcamp/cslage/ats_data/20190416/ptc_craig_config_lights_off/' # BNL/ITL voltages
thedir = '/project/rhl/Data/auxTel/raw/Tucson/ACCS_images/ats/mcm/20190308/' # Older data
%cd $thedir


/project/rhl/Data/auxTel/raw/Tucson/ACCS_images/ats/mcm/20190308


In [10]:
#even_patterns = np.sort(glob.glob('AT_C_20190416_000???_exp_*0.fits'))[0::2] # 20190416
#odd_patterns = np.sort(glob.glob('AT_C_20190416_000???_exp_*0.fits'))[1::2] # 20190416
even_patterns = np.sort(glob.glob('AT_O_20190308_0001??.fits'))[53:71][0::2] # 20190308
odd_patterns = np.sort(glob.glob('AT_O_20190308_0001??.fits'))[53:71][1::2] # 20190308
xmin = 100
xmax = 400
ymin = 700
ymax =1300
numsegments = 16
numfiles = len(even_patterns)
print(len(even_patterns), len(odd_patterns))
print(even_patterns)
print(odd_patterns)

9 9
['AT_O_20190308_000153.fits' 'AT_O_20190308_000155.fits'
 'AT_O_20190308_000157.fits' 'AT_O_20190308_000159.fits'
 'AT_O_20190308_000161.fits' 'AT_O_20190308_000163.fits'
 'AT_O_20190308_000165.fits' 'AT_O_20190308_000167.fits'
 'AT_O_20190308_000169.fits']
['AT_O_20190308_000154.fits' 'AT_O_20190308_000156.fits'
 'AT_O_20190308_000158.fits' 'AT_O_20190308_000160.fits'
 'AT_O_20190308_000162.fits' 'AT_O_20190308_000164.fits'
 'AT_O_20190308_000166.fits' 'AT_O_20190308_000168.fits'
 'AT_O_20190308_000170.fits']


In [11]:
variances = np.zeros([numfiles, numsegments])
avgs = np.zeros([numfiles, numsegments])
segnames = {}
truncated_data = np.zeros([ymax-ymin, xmax-xmin])

for i in range(numfiles):

    if i % 2 == 0:
        file2 = glob.glob(odd_patterns[i])
        file1 = glob.glob(even_patterns[i])
    else:
        file1 = glob.glob(odd_patterns[i])
        file2 = glob.glob(even_patterns[i])

    hdulist1 = pf.open(file2[0], mode='readonly', do_not_scale_image_data=True)
    hdulist2 = pf.open(file1[0], mode='readonly', do_not_scale_image_data=True)
    for segment in range(numsegments):
        
        imhdr1=hdulist1[segment+1].header
        extname = imhdr1['EXTNAME']
        segnames[str(segment)] = extname
        imhdr2=hdulist2[segment+1].header

        data1 = np.array(hdulist1[segment+1].data + 32768, dtype = np.int32)
        data2 = np.array(hdulist2[segment+1].data + 32768, dtype = np.int32)
        
        flat_data1 = np.array(hdulist1[segment+1].data + 32768, dtype = np.int32)[ymin:ymax, xmin:xmax].flatten()
        flat_data2 = np.array(hdulist2[segment+1].data + 32768, dtype = np.int32)[ymin:ymax, xmin:xmax].flatten()
        truncated_data = flat_data2 - flat_data1
        #print truncated_data.min(), truncated_data.max()
        m1 = flat_data1.mean()
        m2 = flat_data2.mean()
        s1 = flat_data1.std()
        s2 = flat_data2.std()
        clipped_truncated_data = []
        for ii in range(len(flat_data1)):
            if (flat_data1[ii] > (m1 - 4.0 * s1)) and (flat_data1[ii] < (m1 + 4.0 * s1))\
            and (flat_data2[ii] > (m2 - 4.0 * s2)) and (flat_data2[ii] < (m2 + 4.0 * s2)):
                clipped_truncated_data.append(truncated_data[ii])
        clipped_truncated_data = np.array(clipped_truncated_data)
            
        variance = clipped_truncated_data.std()**2 / 2.0
        print("Mean of TD = %.2f"%truncated_data.mean())
        overscan1 = data1[2005:2021,:].sum(axis=0) / 16.0
        data1 = data1-overscan1
        overscan2 = data2[2005:2021,:].sum(axis=0) / 16.0
        data2 = data2-overscan2
              
        avg = (np.mean(data1[ymin:ymax,xmin:xmax]) + np.mean(data2[ymin:ymax,xmin:xmax]))/ 2.0 
        variances[i,segment] = variance
        avgs[i,segment] = avg
        print("for pattern %s, segment = %s,var = %.2f, mean = %.2f"%(odd_patterns[i],segnames[str(segment)], variance, avg))
    print("Done with pattern %s."%odd_patterns[i])

Mean of TD = -1.38
for pattern AT_O_20190308_000154.fits, segment = Segment10,var = 175.80, mean = -0.72
Mean of TD = -0.38
for pattern AT_O_20190308_000154.fits, segment = Segment11,var = 139.84, mean = -0.28
Mean of TD = -0.14
for pattern AT_O_20190308_000154.fits, segment = Segment12,var = 148.89, mean = -0.35
Mean of TD = -0.58
for pattern AT_O_20190308_000154.fits, segment = Segment13,var = 172.67, mean = -0.07
Mean of TD = -0.29
for pattern AT_O_20190308_000154.fits, segment = Segment14,var = 177.16, mean = -0.04
Mean of TD = -0.01
for pattern AT_O_20190308_000154.fits, segment = Segment15,var = 182.78, mean = 0.04
Mean of TD = -1.41
for pattern AT_O_20190308_000154.fits, segment = Segment16,var = 202.07, mean = -0.36
Mean of TD = -18.46
for pattern AT_O_20190308_000154.fits, segment = Segment17,var = 157584.15, mean = -10.70
Mean of TD = -2.07
for pattern AT_O_20190308_000154.fits, segment = Segment07,var = 145.77, mean = -1.05
Mean of TD = -1.07
for pattern AT_O_20190308_000154

In [15]:
hdulist = pf.open(even_patterns[0], mode='readonly', do_not_scale_image_data=True)
hdr18=hdulist[18].header
Vph = hdr18['PCK_HIP']
Vpl = hdr18['PCK_LOWP']
Vsh = hdr18['SCK_HIP']
Vsl = hdr18['SCK_LOWP']
Vog = hdr18['OGOV']
Vbb = hdr18['HVBIAS_V']

plt.figure(figsize=(15,15))
plt.subplots_adjust(hspace = 1.2, wspace = 1.2)
plt.suptitle("Gain and Noise by segment - ITL AuxTel CCD 098\n Vph=%.2f; Vpl=%.2f; Vsh=%.2f; Vsl=%.2f; Vog=%.2f; Vbb=%.2f"%(Vph,Vpl,Vsh,Vsl,Vog,Vbb), fontsize = 16)
for segment in range(numsegments):
    plt.subplot(4,4,segment+1)

    ptc_coefs = np.polyfit(avgs[1:-1,segment], variances[1:-1,segment], 2)
    slopeToUse = ptc_coefs[1]
    gain = 1.0 / slopeToUse
    rms_noise = np.sign(ptc_coefs[2]) * np.sqrt(abs(ptc_coefs[2])) * gain
    print("%s, gain = %.2f, noise = %.1f"%(segnames[str(segment)],gain, rms_noise))
    plt.title("%s, \n Gain = %.2f, Noise = %.1f e-"%(segnames[str(segment)],gain, rms_noise), fontsize=12)
    plt.scatter(avgs[1:-1,segment], variances[1:-1,segment])
    xplot = np.linspace(0.0,200000, 500)
    #yplot = ptc_coefs[0]*xplot*xplot*xplot + ptc_coefs[1]*xplot*xplot + ptc_coefs[2]*xplot+ptc_coefs[3]
    yplot = ptc_coefs[0]*xplot*xplot + ptc_coefs[1]*xplot + ptc_coefs[2]
    plt.plot(xplot, yplot, color = 'red')
    plt.ylim(0, 200000)
    plt.yticks([0, 100000],fontsize=16)
    plt.xlim(0,200000)
    plt.xticks([0,100000], fontsize=16)

    plt.xlabel("Signal (ADU)",fontsize = 16)
    plt.ylabel("Var. (ADU^2)", fontsize = 16)

#plt.savefig(thedir+"Gain_AuxTel_16Apr19.pdf")
plt.savefig("/home/cslage/WORK/ats/Gain_AuxTel_20190308_16Apr19.pdf")
plt.clf()

Segment10, gain = 0.72, noise = 12.3
Segment11, gain = 0.72, noise = 10.9
Segment12, gain = 0.73, noise = 12.8
Segment13, gain = 0.71, noise = -5.0
Segment14, gain = 0.71, noise = 7.6
Segment15, gain = 0.71, noise = 9.6
Segment16, gain = 0.73, noise = 12.8
Segment17, gain = 0.44, noise = 165.5
Segment07, gain = 0.75, noise = 4.1
Segment06, gain = 0.73, noise = 8.4
Segment05, gain = 0.75, noise = 11.9
Segment04, gain = 0.74, noise = 13.3
Segment03, gain = 0.72, noise = 11.4
Segment02, gain = 0.73, noise = 11.8
Segment01, gain = 0.72, noise = 8.8
Segment00, gain = 0.73, noise = 11.3
